## Getting Acquainted

https://docs.pola.rs/user-guide/getting-started/

https://docs.pola.rs/api/

## Imports

In [106]:
import polars as pl
import polars.selectors as cs
import pandas as pd
import numpy as np

## I/O

Usual suspects for I/O include parquet, csv, json, etc.

https://docs.pola.rs/user-guide/io/

In [107]:
df_census = pl.read_csv('data/census_income.csv')
# df_census.to_pandas() # if you need your pandas fix

BUT! We can use the Lazy API.

In [108]:
df_0 = (
    pl.scan_csv('data/census_income.csv')
    .with_columns(pl.col('age').cast(pl.Int32))
    .filter(pl.col("age") > 25)
)

It's not executed at this stage.

In [109]:
df_0

<LazyFrame [15 cols, {"age": Int32 … "income": String}] at 0x158C2DD00>

Can convert current to lazy.

In [110]:
df_census.lazy().filter(pl.col("age") > 25)

<LazyFrame [15 cols, {"age": Int64 … "income": String}] at 0x158C3B470>

When ready, just collect.

In [111]:
# https://github.com/pola-rs/polars/issues/11592

# describe is almost always better displayed as transposed with even just a few columns, 
# but currently a bug in polars
# df_census['age'].describe().pivot(columns='statistic', values = 'value', index=None) 

In [112]:
df_0.collect()['age'].describe()

statistic,value
str,f64
"""count""",26150.0
"""null_count""",0.0
"""mean""",42.782256
"""std""",11.856834
"""min""",26.0
"""25%""",33.0
"""50%""",41.0
"""75%""",50.0
"""max""",90.0


## Data Creation

In [113]:
from datetime import datetime

df = pl.DataFrame(
    {
        "integer": [1, 2, 3],
        "date": [
            datetime(2025, 1, 1),
            datetime(2025, 1, 2),
            datetime(2025, 1, 3),
        ],
        "float": [4.0, 5.0, 6.0],
        "string": ["a", "b", "c"],
    }
)

df

integer,date,float,string
i64,datetime[μs],f64,str
1,2025-01-01 00:00:00,4.0,"""a"""
2,2025-01-02 00:00:00,5.0,"""b"""
3,2025-01-03 00:00:00,6.0,"""c"""


Notice anything missing? An index! Yay!

## Data Inspection

In [114]:
df_census.shape

(32561, 15)

In [115]:
df_census.dtypes

[Int64,
 String,
 Int64,
 String,
 Int64,
 String,
 String,
 String,
 String,
 String,
 Int64,
 Int64,
 Int64,
 String,
 String]

Unfortunately it keeps the pandas describe tradition of columns as columns and stats as rows. I'm always doing `describe().T` in pandas to get the columns as rows because scrolling down is easier than across. Unfortunately for polars, a pivot bug and odd (to me) transpose behavior preclude this for now.

In [116]:
df_census.describe()

statistic,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
str,f64,str,f64,str,f64,str,str,str,str,str,f64,f64,f64,str,str
"""count""",32561.0,"""32561""",32561.0,"""32561""",32561.0,"""32561""","""32561""","""32561""","""32561""","""32561""",32561.0,32561.0,32561.0,"""32561""","""32561"""
"""null_count""",0.0,"""0""",0.0,"""0""",0.0,"""0""","""0""","""0""","""0""","""0""",0.0,0.0,0.0,"""0""","""0"""
"""mean""",38.581647,null,189778.366512,null,10.080679,null,null,null,null,null,1077.648844,87.30383,40.437456,null,null
"""std""",13.640433,null,105549.977697,null,2.57272,null,null,null,null,null,7385.292085,402.960219,12.347429,null,null
"""min""",17.0,"""?""",12285.0,"""10th""",1.0,"""Divorced""","""?""","""Husband""","""Amer-Indian-Es…","""Female""",0.0,0.0,1.0,"""?""","""<=50K"""
"""25%""",28.0,null,117827.0,null,9.0,null,null,null,null,null,0.0,0.0,40.0,null,null
"""50%""",37.0,null,178356.0,null,10.0,null,null,null,null,null,0.0,0.0,40.0,null,null
"""75%""",48.0,null,237051.0,null,12.0,null,null,null,null,null,0.0,0.0,45.0,null,null
"""max""",90.0,"""Without-pay""",1.484705e6,"""Some-college""",16.0,"""Widowed""","""Transport-movi…","""Wife""","""White""","""Male""",99999.0,4356.0,99.0,"""Yugoslavia""",""">50K"""


Also, dtype should be an arg to describe.

In [117]:
df_census.select(cs.by_dtype(pl.NUMERIC_DTYPES)).describe()

statistic,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
str,f64,f64,f64,f64,f64,f64
"""count""",32561.0,32561.0,32561.0,32561.0,32561.0,32561.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",38.581647,189778.366512,10.080679,1077.648844,87.30383,40.437456
"""std""",13.640433,105549.977697,2.57272,7385.292085,402.960219,12.347429
"""min""",17.0,12285.0,1.0,0.0,0.0,1.0
"""25%""",28.0,117827.0,9.0,0.0,0.0,40.0
"""50%""",37.0,178356.0,10.0,0.0,0.0,40.0
"""75%""",48.0,237051.0,12.0,0.0,0.0,45.0
"""max""",90.0,1.484705e6,16.0,99999.0,4356.0,99.0


In [118]:
df_census.null_count()  # come back after replacing ? with null

age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [119]:
df_census.select(pl.all().n_unique())

age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
73,9,21648,16,16,7,15,6,5,2,119,92,94,42,2


In [120]:
df_census.count() # come back after replacing ? with null

age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
32561,32561,32561,32561,32561,32561,32561,32561,32561,32561,32561,32561,32561,32561,32561


## Selection and Filtering

### Selection

https://docs.pola.rs/user-guide/expressions/column-selections/

We can think of selection in two ways. We can think of it as filtering columns to those selected, or choosing which columns to work on to manipulate the data. The basic `.select` method is used primarily for the former, and the `.with_columns` function is for the latter, which we'll see later.

In [121]:
df_census.select("age", "education").head()

age,education
i64,str
90,"""HS-grad"""
82,"""HS-grad"""
66,"""Some-college"""
54,"""7th-8th"""
41,"""Some-college"""


In [122]:
df_census[['age', 'education']].head() # still fine, but no reason to do this.

age,education
i64,str
90,"""HS-grad"""
82,"""HS-grad"""
66,"""Some-college"""
54,"""7th-8th"""
41,"""Some-college"""


Use selectors to make things easier!

https://docs.pola.rs/py-polars/html/reference/selectors.html

https://docs.pola.rs/user-guide/expressions/column-selections/#using-selectors

In [123]:
df_census.select(cs.string()).dtypes

[String, String, String, String, String, String, String, String, String]

In [124]:
df_census.select(cs.starts_with("capital"), cs.matches('educ|inc'))

capital.gain,capital.loss,education,education.num,income
i64,i64,str,i64,str
0,4356,"""HS-grad""",9,"""<=50K"""
0,4356,"""HS-grad""",9,"""<=50K"""
0,4356,"""Some-college""",10,"""<=50K"""
0,3900,"""7th-8th""",4,"""<=50K"""
0,3900,"""Some-college""",10,"""<=50K"""
0,3770,"""HS-grad""",9,"""<=50K"""
0,3770,"""10th""",6,"""<=50K"""
0,3683,"""Doctorate""",16,""">50K"""
0,3683,"""HS-grad""",9,"""<=50K"""


In [125]:
df_census.select(cs.numeric())

age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
i64,i64,i64,i64,i64,i64
90,77053,9,0,4356,40
82,132870,9,0,4356,18
66,186061,10,0,4356,40
54,140359,4,0,3900,40
41,264663,10,0,3900,40
34,216864,9,0,3770,45
38,150601,6,0,3770,40
74,88638,16,0,3683,20
68,422013,9,0,3683,40


In [126]:
df_census.select(cs.exclude(cs.string()))

age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
i64,i64,i64,i64,i64,i64
90,77053,9,0,4356,40
82,132870,9,0,4356,18
66,186061,10,0,4356,40
54,140359,4,0,3900,40
41,264663,10,0,3900,40
34,216864,9,0,3770,45
38,150601,6,0,3770,40
74,88638,16,0,3683,20
68,422013,9,0,3683,40


In [127]:
df_census.select(cs.string() - cs.last()) # all strings but the last one

workclass,education,marital.status,occupation,relationship,race,sex,native.country
str,str,str,str,str,str,str,str
"""?""","""HS-grad""","""Widowed""","""?""","""Not-in-family""","""White""","""Female""","""United-States"""
"""Private""","""HS-grad""","""Widowed""","""Exec-manageria…","""Not-in-family""","""White""","""Female""","""United-States"""
"""?""","""Some-college""","""Widowed""","""?""","""Unmarried""","""Black""","""Female""","""United-States"""
"""Private""","""7th-8th""","""Divorced""","""Machine-op-ins…","""Unmarried""","""White""","""Female""","""United-States"""
"""Private""","""Some-college""","""Separated""","""Prof-specialty…","""Own-child""","""White""","""Female""","""United-States"""
"""Private""","""HS-grad""","""Divorced""","""Other-service""","""Unmarried""","""White""","""Female""","""United-States"""
"""Private""","""10th""","""Separated""","""Adm-clerical""","""Unmarried""","""White""","""Male""","""United-States"""
"""State-gov""","""Doctorate""","""Never-married""","""Prof-specialty…","""Other-relative…","""White""","""Female""","""United-States"""
"""Federal-gov""","""HS-grad""","""Divorced""","""Prof-specialty…","""Not-in-family""","""White""","""Female""","""United-States"""


Lots of handy selectors.

In [128]:
df.select(cs.temporal()) # also date, datetime, etc.

date
datetime[μs]
2025-01-01 00:00:00
2025-01-02 00:00:00
2025-01-03 00:00:00


### Filtering

The nice thing about Polars is that it doesn't conflate filtering with selection, and also the lack of index means we don't need additional/special methods to get around it. No `.loc`, `.iloc`, for example. 

In [129]:
(
    df_census
    .filter(pl.col("age") >= 90, pl.col('hours.per.week') >= 40) # wtf are these presidential candidates or something?
)

age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
i64,str,i64,str,i64,str,str,str,str,str,i64,i64,i64,str,str
90,"""?""",77053,"""HS-grad""",9,"""Widowed""","""?""","""Not-in-family""","""White""","""Female""",0,4356,40,"""United-States""","""<=50K"""
90,"""Private""",51744,"""HS-grad""",9,"""Never-married""","""Other-service""","""Not-in-family""","""Black""","""Male""",0,2206,40,"""United-States""","""<=50K"""
90,"""Local-gov""",227796,"""Masters""",14,"""Married-civ-sp…","""Exec-manageria…","""Husband""","""White""","""Male""",20051,0,60,"""United-States""",""">50K"""
90,"""Private""",87372,"""Prof-school""",15,"""Married-civ-sp…","""Prof-specialty…","""Husband""","""White""","""Male""",20051,0,72,"""United-States""",""">50K"""
90,"""Self-emp-not-i…",155981,"""Bachelors""",13,"""Married-civ-sp…","""Prof-specialty…","""Husband""","""White""","""Male""",10566,0,50,"""United-States""","""<=50K"""
90,"""Private""",175491,"""HS-grad""",9,"""Married-civ-sp…","""Craft-repair""","""Husband""","""White""","""Male""",9386,0,50,"""Ecuador""",""">50K"""
90,"""Local-gov""",153602,"""HS-grad""",9,"""Married-civ-sp…","""Other-service""","""Husband""","""White""","""Male""",6767,0,40,"""United-States""","""<=50K"""
90,"""Local-gov""",214594,"""7th-8th""",4,"""Married-civ-sp…","""Protective-ser…","""Husband""","""White""","""Male""",2653,0,40,"""United-States""","""<=50K"""
90,"""Private""",137018,"""HS-grad""",9,"""Never-married""","""Other-service""","""Not-in-family""","""White""","""Female""",0,0,40,"""United-States""","""<=50K"""


In [130]:
(
    df_census
    .with_columns(index = np.arange(0, df_census.shape[0]))
    .slice(7, 3) # start, length
    .select('index', cs.all()-cs.last()) # last is 'index' and 'index' is just a normal column
)

index,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
i64,i64,str,i64,str,i64,str,str,str,str,str,i64,i64,i64,str,str
7,74,"""State-gov""",88638,"""Doctorate""",16,"""Never-married""","""Prof-specialty…","""Other-relative…","""White""","""Female""",0,3683,20,"""United-States""",""">50K"""
8,68,"""Federal-gov""",422013,"""HS-grad""",9,"""Divorced""","""Prof-specialty…","""Not-in-family""","""White""","""Female""",0,3683,40,"""United-States""","""<=50K"""
9,41,"""Private""",70037,"""Some-college""",10,"""Never-married""","""Craft-repair""","""Unmarried""","""White""","""Male""",0,3004,60,"""?""",""">50K"""


## Creating New/Transforming Features

Wherein we waste a lot of programming time relative to tidyverse...

Here we create a couple new columns based on existing ones in the DataFrame.

In [131]:
(
    df_census.with_columns(
        capital_total=pl.sum_horizontal(['capital.gain', 'capital.loss']),
        capital_any = pl.any_horizontal(['capital.gain', 'capital.loss']),
        # here we would ideally use polars functions for optimized performance, e.g.
        # as opposed to created a function
        age_sc = (pl.col('age') - pl.col('age').mean()) / pl.col('age').std()
    )
    .select(cs.matches('^capital|^age'))
)

age,capital.gain,capital.loss,capital_total,capital_any,age_sc
i64,i64,i64,i64,bool,f64
90,0,4356,4356,true,3.769554
82,0,4356,4356,true,3.183063
66,0,4356,4356,true,2.010079
54,0,3900,3900,true,1.130342
41,0,3900,3900,true,0.177293
34,0,3770,3770,true,-0.335887
38,0,3770,3770,true,-0.042641
74,0,3683,3683,true,2.596571
68,0,3683,3683,true,2.156702


If you are hell-bent on doing things the hard way...

In [132]:
df_census.select(pl.sql_expr('MAX(age + 1)'))

age
i64
91


### Categoricals

Categoricals can use `Enum` or `Categorical` dtype. If you know the categories, use `Enum`. If you think they might change, use `Categorical`. But you can cast from one to the other.

Some apply on selection approaches are clunky at best.

In [133]:
df_census = df_census.with_columns(
    pl.when(cs.string() != '?').then(cs.string()).otherwise(None) # this works but is not what anyone would guess
    # pl.when(cs.string() == '?').then(None).otherwise(cs.string()) # doesn't work
)


In [134]:
df_census.with_columns(cs.string().cast(pl.Categorical)) # very nice

age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
i64,cat,i64,cat,i64,cat,cat,cat,cat,cat,i64,i64,i64,cat,cat
90,null,77053,"""HS-grad""",9,"""Widowed""",null,"""Not-in-family""","""White""","""Female""",0,4356,40,"""United-States""","""<=50K"""
82,"""Private""",132870,"""HS-grad""",9,"""Widowed""","""Exec-manageria…","""Not-in-family""","""White""","""Female""",0,4356,18,"""United-States""","""<=50K"""
66,null,186061,"""Some-college""",10,"""Widowed""",null,"""Unmarried""","""Black""","""Female""",0,4356,40,"""United-States""","""<=50K"""
54,"""Private""",140359,"""7th-8th""",4,"""Divorced""","""Machine-op-ins…","""Unmarried""","""White""","""Female""",0,3900,40,"""United-States""","""<=50K"""
41,"""Private""",264663,"""Some-college""",10,"""Separated""","""Prof-specialty…","""Own-child""","""White""","""Female""",0,3900,40,"""United-States""","""<=50K"""
34,"""Private""",216864,"""HS-grad""",9,"""Divorced""","""Other-service""","""Unmarried""","""White""","""Female""",0,3770,45,"""United-States""","""<=50K"""
38,"""Private""",150601,"""10th""",6,"""Separated""","""Adm-clerical""","""Unmarried""","""White""","""Male""",0,3770,40,"""United-States""","""<=50K"""
74,"""State-gov""",88638,"""Doctorate""",16,"""Never-married""","""Prof-specialty…","""Other-relative…","""White""","""Female""",0,3683,20,"""United-States""",""">50K"""
68,"""Federal-gov""",422013,"""HS-grad""",9,"""Divorced""","""Prof-specialty…","""Not-in-family""","""White""","""Female""",0,3683,40,"""United-States""","""<=50K"""


Unlike pandas, we don't need a lambda for everything.

In [135]:
(
    df_census
    .with_columns(
        log_capital_gain = pl.col('capital.gain').log1p(),
        fulltime = pl.when(pl.col('hours.per.week') >= 40).then(True).otherwise(False)
    )
    .select(cs.matches('capital|hours|fulltime'))
)

capital.gain,capital.loss,hours.per.week,log_capital_gain,fulltime
i64,i64,i64,f64,bool
0,4356,40,0.0,true
0,4356,18,0.0,false
0,4356,40,0.0,true
0,3900,40,0.0,true
0,3900,40,0.0,true
0,3770,45,0.0,true
0,3770,40,0.0,true
0,3683,20,0.0,false
0,3683,40,0.0,true


For more complicated approaches you might want to investigate [folds](https://docs.pola.rs/user-guide/expressions/folds/).

## Group & Windowed Operations

One of the greatest improvements in polars over pandas is using snake case instead of the inappropriate camel case for `group_by`. It's the little things. 🐍  

Here is a simple group operation done two different ways. Pretty straightforward.

In [136]:
df_census.group_by('workclass').agg(mean_hours_per_week = pl.col('hours.per.week').mean())  # either is fine
df_census.group_by('workclass').agg(pl.mean('hours.per.week').alias('mean_hours_per_week'))

workclass,mean_hours_per_week
str,f64
"""Never-worked""",28.428571
"""Without-pay""",32.714286
"""State-gov""",39.031587
"""Private""",40.267096
"""Local-gov""",40.9828
"""Self-emp-inc""",48.8181
"""Self-emp-not-i…",44.421881
"""Federal-gov""",41.379167
null,31.91939


The two above approaches will not return the same order of results, and actually the returned order is not guaranteed at all. If you need to guarantee the order (i.e. most of the time), you can use  the `maintain_order` arg (and take a speed hit for very large data) or `sort` the result after. 

In [137]:
df_census.group_by('workclass').agg(mean_hours_per_week = pl.col('hours.per.week').mean()).sort('workclass')
df_census.group_by('workclass', maintain_order=True).agg(pl.mean('hours.per.week').alias('mean_hours_per_week'))

workclass,mean_hours_per_week
str,f64
null,31.91939
"""Private""",40.267096
"""State-gov""",39.031587
"""Federal-gov""",41.379167
"""Self-emp-not-i…",44.421881
"""Self-emp-inc""",48.8181
"""Local-gov""",40.9828
"""Without-pay""",32.714286
"""Never-worked""",28.428571


I want to see the unique values of education.num for each education category. Since we wouldn't know in a lot of situations that the result is a single value, we would the list of unique values as a **list column**, which can then be explored as such. In this case we can just take the first/only value.

In [138]:
(
    df_census
    .group_by('education')
    .agg(pl.col('education.num').unique()) # add .first() to get the first value
)

education,education.num
str,list[i64]
"""1st-4th""",[2]
"""Prof-school""",[15]
"""Assoc-voc""",[11]
"""12th""",[8]
"""Bachelors""",[13]
"""10th""",[6]
"""Assoc-acdm""",[12]
"""Some-college""",[10]
"""5th-6th""",[3]


Given the above, we can group by college vs. not and get the proportion of those with a salary > 50k. I left a couple issues I came across when doing this.

In [139]:
(
    df_census
    .with_columns(
        # when requires same dtype; and honestly, is not as intuitive as it should be
        # pl.when(pl.col('education.num') >= 10).then('Some college').otherwise('HS or less') # doesn't work because of different type- boo!
        pl.when(pl.col('education.num') >= 10).then(1).otherwise(0)
        .alias('college')
        # can't cast numerics to categorical, so you'd have to break it into two steps to create categorical with levels
        # .cast(pl.String) 
        # .cast(pl.Categorical)
    )
    .group_by('college')
    .agg((pl.col('income') == '>50K').drop_nulls().mean()) # annoying, no arg for drop NA w/in mean
)

college,income
i32,f64
0,0.130066
1,0.332566


The following represents a very "tidy" approach. 

In [140]:
(
    df_census
    .group_by('education')
    .agg(
        FT = (pl.col("hours.per.week") >=40).sum(),
        PT = (pl.col("hours.per.week") < 40).sum(),
        prop_FT = (pl.col("hours.per.week") >=40).drop_nans().mean()
    )
    .limit(5)
)

education,FT,PT,prop_FT
str,u32,u32,f64
"""HS-grad""",8223,2278,0.783068
"""Prof-school""",480,96,0.833333
"""Doctorate""",352,61,0.8523
"""Preschool""",31,20,0.607843
"""12th""",270,163,0.623557


What if we want a group operation that maintains the original DataFrame shape? Here is one method using the group_by approach.

In [141]:
(
    df_census
    .group_by('workclass')
    .agg(hours_centered = pl.col('hours.per.week') -  pl.col('hours.per.week').mean())
    .explode('hours_centered')
)

workclass,hours_centered
str,f64
null,8.08061
null,8.08061
null,8.08061
null,0.08061
null,-16.91939
null,8.08061
null,8.08061
null,-19.91939
null,-25.91939


In [142]:
(
    df_census
    .group_by('workclass')
    .agg(
        age_x = pl.map_groups(['age'], function = lambda x: x[0] - x[0].mean())
    )
)

workclass,age_x
str,list[f64]
"""Self-emp-not-i…","[-6.969697, 15.030303, … -1.969697]"
"""Local-gov""","[-1.751075, -0.751075, … -2.751075]"
"""State-gov""","[34.563945, 11.563945, … 3.563945]"
null,"[49.03976, 25.03976, … 31.03976]"
"""Without-pay""","[-25.785714, 17.214286, … 14.214286]"
"""Self-emp-inc""","[-9.017025, 12.982975, … 11.982975]"
"""Private""","[45.202415, 17.202415, … -14.797585]"
"""Federal-gov""","[25.409375, 20.409375, … -3.590625]"
"""Never-worked""","[-2.571429, 2.428571, … -2.571429]"


### Windowed Operations

The above is fine, and working with list columns is very handy for many settings, but we didn't really want to aggregate, which would require us to rejoin to the original frame, we just wanted to add a column to the existing data frame, or create a new subset of the same number of rows.  Pandas is very clunky or slow in this regard, but polars is better. Its windowed operations allow us to perform aggregations on groups in the select context.

Here we subtract the group mean from hours per week.

In [143]:
(
    df_census
    .select(
        'workclass', 
        'hours.per.week',
        hours_centered = (pl.col('hours.per.week') -  pl.col('hours.per.week').mean()).over('workclass')
    )
)

workclass,hours.per.week,hours_centered
str,i64,f64
null,40,8.08061
"""Private""",18,-22.267096
null,40,8.08061
"""Private""",40,-0.267096
"""Private""",40,-0.267096
"""Private""",45,4.732904
"""Private""",40,-0.267096
"""State-gov""",20,-19.031587
"""Federal-gov""",40,-1.379167


In [144]:
# using transform with a lambda on large data is unreasonably slow, and now you have to merge it back in or split out this operation to create the column and hope there aren't any indexing issues
# df_census.to_pandas().groupby('workclass')['hours.per.week'].transform(lambda x: x - x.mean()) 

Of course, still nowhere near the simplicity of the tidyverse, but take what you can get. 😄

In [145]:
# No quotes necessary, not even a group_by!
# df_census |>
#     mutate(
#         hours_centered = hours.per.week - mean(hours.per.week),
#         .by = workclass
#     )

Some more examples

In [146]:
df_pokemon = pl.read_csv(
    "https://gist.githubusercontent.com/ritchie46/cac6b337ea52281aa23c049250a4ff03/raw/89a957ff3919d90e6ef2d34235e6bf22304f3366/pokemon.csv"
)

df_pokemon.head(), df_pokemon.shape

(shape: (5, 13)
 ┌─────┬───────────────────────┬────────┬────────┬───┬─────────┬───────┬────────────┬───────────┐
 │ #   ┆ Name                  ┆ Type 1 ┆ Type 2 ┆ … ┆ Sp. Def ┆ Speed ┆ Generation ┆ Legendary │
 │ --- ┆ ---                   ┆ ---    ┆ ---    ┆   ┆ ---     ┆ ---   ┆ ---        ┆ ---       │
 │ i64 ┆ str                   ┆ str    ┆ str    ┆   ┆ i64     ┆ i64   ┆ i64        ┆ bool      │
 ╞═════╪═══════════════════════╪════════╪════════╪═══╪═════════╪═══════╪════════════╪═══════════╡
 │ 1   ┆ Bulbasaur             ┆ Grass  ┆ Poison ┆ … ┆ 65      ┆ 45    ┆ 1          ┆ false     │
 │ 2   ┆ Ivysaur               ┆ Grass  ┆ Poison ┆ … ┆ 80      ┆ 60    ┆ 1          ┆ false     │
 │ 3   ┆ Venusaur              ┆ Grass  ┆ Poison ┆ … ┆ 100     ┆ 80    ┆ 1          ┆ false     │
 │ 3   ┆ VenusaurMega Venusaur ┆ Grass  ┆ Poison ┆ … ┆ 120     ┆ 80    ┆ 1          ┆ false     │
 │ 4   ┆ Charmander            ┆ Fire   ┆ null   ┆ … ┆ 50      ┆ 65    ┆ 1          ┆ false     │
 └──

In [147]:
(
    df_pokemon
    .select(
        cs.exclude('Generation', 'Legendary'),
        attack2 = pl.col('Attack').mean().over(['Type 1', 'Type 2']),
    )
)

#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,attack2
i64,str,str,str,i64,i64,i64,i64,i64,i64,i64,f64
1,"""Bulbasaur""","""Grass""","""Poison""",318,45,49,49,65,65,45,75.8
2,"""Ivysaur""","""Grass""","""Poison""",405,60,62,63,80,80,60,75.8
3,"""Venusaur""","""Grass""","""Poison""",525,80,82,83,100,100,80,75.8
3,"""VenusaurMega V…","""Grass""","""Poison""",625,80,100,123,122,120,80,75.8
4,"""Charmander""","""Fire""",null,309,39,52,43,60,50,65,82.3
5,"""Charmeleon""","""Fire""",null,405,58,64,58,80,65,80,82.3
6,"""Charizard""","""Fire""","""Flying""",534,78,84,78,109,85,100,96.0
6,"""CharizardMega …","""Fire""","""Dragon""",634,78,130,111,130,85,100,130.0
6,"""CharizardMega …","""Fire""","""Flying""",634,78,104,78,159,115,100,96.0


In [148]:
(
    df_pokemon.select(
        pl.col('Name', 'Type 1', 'Type 2', 'HP')
        .sort_by('HP', descending=True)
        .over((['Type 1', 'Type 2']))
    )
)

Name,Type 1,Type 2,HP
str,str,str,i64
"""Venusaur""","""Grass""","""Poison""",80
"""VenusaurMega V…","""Grass""","""Poison""",80
"""Victreebel""","""Grass""","""Poison""",80
"""Vileplume""","""Grass""","""Poison""",75
"""Arcanine""","""Fire""",null,90
"""Ninetales""","""Fire""",null,73
"""Moltres""","""Fire""","""Flying""",90
"""CharizardMega …","""Fire""","""Dragon""",78
"""Charizard""","""Fire""","""Flying""",78


## Joins

Polars, minimizes data expansion to two sensible approaches:
- concat: for stacking dataframes 
- join: for merging dataframes

Like pandas, polars can't simply refer to 'rows' and 'columns' (even though it is explicitly 2D).

In [149]:
df_2 = pl.DataFrame(
    {
        "a": [1, 2, 3],
        "b": [4, 5, 6],
        "c": [7, 8, 9],
    }
)

df_3 = pl.DataFrame({
    "integer": [1, 2,3],
    "date": [datetime(2025, 1, 1), datetime(2025, 1, 2), datetime(2025, 1, 3)],
    "float": [7.0, 8.0, 9.0],
    "string": ["d", "e", "f"],
})

df_4 = df_2.with_columns(string = np.array(['a', 'b', 'd']))




In [150]:
pl.concat([df, df_2], how = "horizontal")

integer,date,float,string,a,b,c
i64,datetime[μs],f64,str,i64,i64,i64
1,2025-01-01 00:00:00,4.0,"""a""",1,4,7
2,2025-01-02 00:00:00,5.0,"""b""",2,5,8
3,2025-01-03 00:00:00,6.0,"""c""",3,6,9


In [151]:
pl.concat([df, df_3], how = "vertical")

integer,date,float,string
i64,datetime[μs],f64,str
1,2025-01-01 00:00:00,4.0,"""a"""
2,2025-01-02 00:00:00,5.0,"""b"""
3,2025-01-03 00:00:00,6.0,"""c"""
1,2025-01-01 00:00:00,7.0,"""d"""
2,2025-01-02 00:00:00,8.0,"""e"""
3,2025-01-03 00:00:00,9.0,"""f"""


In [152]:
df.join(df_4, on = 'string', how = 'inner')

integer,date,float,string,a,b,c
i64,datetime[μs],f64,str,i64,i64,i64
1,2025-01-01 00:00:00,4.0,"""a""",1,4,7
2,2025-01-02 00:00:00,5.0,"""b""",2,5,8


In [153]:
df.join(df_4, on = 'string', how = 'left')

integer,date,float,string,a,b,c
i64,datetime[μs],f64,str,i64,i64,i64
1,2025-01-01 00:00:00,4.0,"""a""",1,4,7
2,2025-01-02 00:00:00,5.0,"""b""",2,5,8
3,2025-01-03 00:00:00,6.0,"""c""",null,null,null


There is also `hstack` and `vstack` but who knows why.

## Reshape/Transformations

### Pivots

In [154]:
df_pivot = pl.concat([df, df_3], how = "vertical")
df_pivot

integer,date,float,string
i64,datetime[μs],f64,str
1,2025-01-01 00:00:00,4.0,"""a"""
2,2025-01-02 00:00:00,5.0,"""b"""
3,2025-01-03 00:00:00,6.0,"""c"""
1,2025-01-01 00:00:00,7.0,"""d"""
2,2025-01-02 00:00:00,8.0,"""e"""
3,2025-01-03 00:00:00,9.0,"""f"""


In [155]:
df_pivot.pivot(index = 'date', columns = 'string', values = 'integer')

date,a,b,c,d,e,f
datetime[μs],i64,i64,i64,i64,i64,i64
2025-01-01 00:00:00,1,null,null,1,null,null
2025-01-02 00:00:00,null,2,null,null,2,null
2025-01-03 00:00:00,null,null,3,null,null,3


In [156]:
df_pivot = pl.concat([df, df_3], how = "vertical")
df_pivot

integer,date,float,string
i64,datetime[μs],f64,str
1,2025-01-01 00:00:00,4.0,"""a"""
2,2025-01-02 00:00:00,5.0,"""b"""
3,2025-01-03 00:00:00,6.0,"""c"""
1,2025-01-01 00:00:00,7.0,"""d"""
2,2025-01-02 00:00:00,8.0,"""e"""
3,2025-01-03 00:00:00,9.0,"""f"""


In [157]:
df_wide = df_pivot.pivot(index = 'date', columns = 'integer', values = 'float', aggregate_function="sum")

df_wide

date,1,2,3
datetime[μs],f64,f64,f64
2025-01-01 00:00:00,11.0,null,null
2025-01-02 00:00:00,null,13.0,null
2025-01-03 00:00:00,null,null,15.0


Aside, the columns above are strings as they should be, not actual numbers.

In [158]:
df_wide.columns

['date', '1', '2', '3']

There is also `unstack` but it is currently 'unstable' and subject to change.

### Melts

If your 'make wide' function is `pivot`, it doesn't make sense to call the 'make long' function `melt`, but I digress. In any case, melt takes data from wide to long format, and unlike pivot, works for lazy dataframes also.

In [159]:
df_wide.melt(id_vars = ['date'], value_vars = ['1', '2', '3'], value_name = 'sum').drop_nulls()

date,variable,sum
datetime[μs],str,f64
2025-01-01 00:00:00,"""1""",11.0
2025-01-02 00:00:00,"""2""",13.0
2025-01-03 00:00:00,"""3""",15.0


## Missing Data

Missing data is *always* `null` in polars, regardless of dtype. Yay!

In [160]:
(
    df_census
    .group_by(pl.col('workclass').fill_null('Other'))
    .count()
    .sort('count', descending=True)
)

/var/folders/x6/4jhswqxj0sqf_gkgq6lw6l880000gn/T/ipykernel_43916/472883945.py:4: DeprecationWarning: `count` is deprecated. It has been renamed to `len`.
  .count()


workclass,count
str,u32
"""Private""",22696
"""Self-emp-not-i…",2541
"""Local-gov""",2093
"""Other""",1836
"""State-gov""",1298
"""Self-emp-inc""",1116
"""Federal-gov""",960
"""Without-pay""",14
"""Never-worked""",7


numpy NaN values can be used, but are considered to be a type of floating point data in Polars. MC sees no obvious benefit to using NaN over null, and there would be potential drawbacks as they are not counted as null, would have to be replaced with null to do a mean or similar function, etc.

## Time Series

When it comes to times series, polars can do all the usual functions like parse-on-read, `str.to_date`, extracting month/year/day from a date column, etc.

In [161]:
df_aq = pl.read_csv("data/air_quality_no2_long.csv", try_parse_dates=True)
df_aq = df_aq.sort("date.utc")
df_aq

city,country,date.utc,location,parameter,value,unit
str,str,"datetime[μs, UTC]",str,str,f64,str
"""Paris""","""FR""",2019-05-07 01:00:00 UTC,"""FR04014""","""no2""",25.0,"""µg/m³"""
"""Antwerpen""","""BE""",2019-05-07 01:00:00 UTC,"""BETR801""","""no2""",50.5,"""µg/m³"""
"""London""","""GB""",2019-05-07 01:00:00 UTC,"""London Westmin…","""no2""",23.0,"""µg/m³"""
"""Paris""","""FR""",2019-05-07 02:00:00 UTC,"""FR04014""","""no2""",27.7,"""µg/m³"""
"""Antwerpen""","""BE""",2019-05-07 02:00:00 UTC,"""BETR801""","""no2""",45.0,"""µg/m³"""
"""London""","""GB""",2019-05-07 02:00:00 UTC,"""London Westmin…","""no2""",19.0,"""µg/m³"""
"""Paris""","""FR""",2019-05-07 03:00:00 UTC,"""FR04014""","""no2""",50.4,"""µg/m³"""
"""London""","""GB""",2019-05-07 03:00:00 UTC,"""London Westmin…","""no2""",19.0,"""µg/m³"""
"""Paris""","""FR""",2019-05-07 04:00:00 UTC,"""FR04014""","""no2""",61.9,"""µg/m³"""


In [162]:
# df_aq.plot('date.utc', 'value')  # requires hvplot

In [163]:
df_aq.filter(
    # pl.col("date.utc").dt.date() == datetime(2019, 6, 20), 
    pl.col("date.utc").dt.month() == 6
)

city,country,date.utc,location,parameter,value,unit
str,str,"datetime[μs, UTC]",str,str,f64,str
"""Paris""","""FR""",2019-06-01 00:00:00 UTC,"""FR04014""","""no2""",84.7,"""µg/m³"""
"""London""","""GB""",2019-06-01 00:00:00 UTC,"""London Westmin…","""no2""",22.0,"""µg/m³"""
"""Paris""","""FR""",2019-06-01 01:00:00 UTC,"""FR04014""","""no2""",74.8,"""µg/m³"""
"""Antwerpen""","""BE""",2019-06-01 01:00:00 UTC,"""BETR801""","""no2""",52.5,"""µg/m³"""
"""London""","""GB""",2019-06-01 01:00:00 UTC,"""London Westmin…","""no2""",22.0,"""µg/m³"""
"""Paris""","""FR""",2019-06-01 02:00:00 UTC,"""FR04014""","""no2""",68.1,"""µg/m³"""
"""London""","""GB""",2019-06-01 02:00:00 UTC,"""London Westmin…","""no2""",16.0,"""µg/m³"""
"""London""","""GB""",2019-06-01 03:00:00 UTC,"""London Westmin…","""no2""",16.0,"""µg/m³"""
"""London""","""GB""",2019-06-01 04:00:00 UTC,"""London Westmin…","""no2""",11.0,"""µg/m³"""


In [164]:
df_aq.with_columns(mo = pl.col("date.utc").dt.month())

city,country,date.utc,location,parameter,value,unit,mo
str,str,"datetime[μs, UTC]",str,str,f64,str,i8
"""Paris""","""FR""",2019-05-07 01:00:00 UTC,"""FR04014""","""no2""",25.0,"""µg/m³""",5
"""Antwerpen""","""BE""",2019-05-07 01:00:00 UTC,"""BETR801""","""no2""",50.5,"""µg/m³""",5
"""London""","""GB""",2019-05-07 01:00:00 UTC,"""London Westmin…","""no2""",23.0,"""µg/m³""",5
"""Paris""","""FR""",2019-05-07 02:00:00 UTC,"""FR04014""","""no2""",27.7,"""µg/m³""",5
"""Antwerpen""","""BE""",2019-05-07 02:00:00 UTC,"""BETR801""","""no2""",45.0,"""µg/m³""",5
"""London""","""GB""",2019-05-07 02:00:00 UTC,"""London Westmin…","""no2""",19.0,"""µg/m³""",5
"""Paris""","""FR""",2019-05-07 03:00:00 UTC,"""FR04014""","""no2""",50.4,"""µg/m³""",5
"""London""","""GB""",2019-05-07 03:00:00 UTC,"""London Westmin…","""no2""",19.0,"""µg/m³""",5
"""Paris""","""FR""",2019-05-07 04:00:00 UTC,"""FR04014""","""no2""",61.9,"""µg/m³""",5


Special functionality for grouped operations.

In [165]:
(
    df_aq
    .with_columns(
        year = pl.col("date.utc").dt.year(),
        month = pl.col("date.utc").dt.month(),
        day = pl.col("date.utc").dt.day(), 
    )
    .group_by_dynamic("date.utc", every = "12h")
    .agg(
        max = pl.col("value").max(),
        min = pl.col("value").min(),
    )
)
    

date.utc,max,min
"datetime[μs, UTC]",f64,f64
2019-05-07 00:00:00 UTC,77.7,13.0
2019-05-07 12:00:00 UTC,36.2,10.6
2019-05-08 00:00:00 UTC,36.0,13.5
2019-05-08 12:00:00 UTC,48.9,14.3
2019-05-09 00:00:00 UTC,50.7,10.0
2019-05-09 12:00:00 UTC,97.0,21.3
2019-05-10 00:00:00 UTC,60.7,10.5
2019-05-10 12:00:00 UTC,43.6,22.0
2019-05-11 00:00:00 UTC,36.8,14.9
